# Comparação de Modelos de Machine Learning para predição de Sobrevivência

In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression

## 1. Tratamento de Dados Discrepantes (Outliers)


In [4]:
df = pd.read_csv('../data/raw/train.csv')
df.to_csv('../data/raw/train.csv', index=False)

In [8]:
# Tratamento de Outliers
Q1 = df.quantile(0.25, numeric_only=True)
Q3 = df.quantile(0.75, numeric_only=True)
IQR = Q3 - Q1

# Limites para identificar outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Alinhar índices e colunas para as comparações
df_alinhado, limite_inferior = df.align(limite_inferior, axis=1, copy=False)
_, limite_superior = df.align(limite_superior, axis=1, copy=False)

# Remover outliers com alinhamento explícito
df_sem_outliers = df_alinhado[~((df_alinhado < limite_inferior) | (df_alinhado > limite_superior)).any(axis=1)]

# Identificar outliers (caso necessário)
outliers = df_alinhado[((df_alinhado < limite_inferior) | (df_alinhado > limite_superior)).any(axis=1)]

print(f"Linhas removidas devido a outliers: {len(df) - len(df_sem_outliers)}")

# Atualizar DataFrame após remoção
df = df_sem_outliers


Linhas removidas devido a outliers: 56


## 2. Tratamento de Dados Faltantes


### * Remover Linhas/Colunas com Dados Faltantes Insignificantes

In [13]:
# Verificar quantidade de dados faltantes por coluna
print(df.isnull().sum())

# Remover colunas com mais de 50% de valores faltantes
df_limpo = df.dropna(thresh=df.shape[0] * 0.5, axis=1).copy()

# Tratamento específico para valores ausentes em 'Age'
# Preenchendo com a mediana, pois 'Age' é uma variável contínua
if 'Age' in df_limpo.columns:
    df_limpo.loc[:, 'Age'] = df_limpo['Age'].fillna(df_limpo['Age'].median())

# Tratamento específico para valores ausentes em 'Embarked'
# Preenchendo com o valor mais frequente (moda)
if 'Embarked' in df_limpo.columns:
    df_limpo.loc[:, 'Embarked'] = df_limpo['Embarked'].fillna(df_limpo['Embarked'].mode()[0])

# Verificar novamente os valores faltantes após os tratamentos
print("Após os tratamentos:")
print(df_limpo.isnull().sum())


Age             99
Cabin          369
Embarked         0
Fare             0
Name             0
Parch            0
PassengerId      0
Pclass           0
Sex              0
SibSp            0
Survived         0
Ticket           0
dtype: int64
Após os tratamentos:
Age            0
Embarked       0
Fare           0
Name           0
Parch          0
PassengerId    0
Pclass         0
Sex            0
SibSp          0
Survived       0
Ticket         0
dtype: int64


### * Imputar Valores Ausentes


In [22]:
# Certifique-se de estar trabalhando com o DataFrame completo e não uma fatia
df = df.copy()  # Garantir que é uma cópia do DataFrame original

# Imputar valores numéricos com a média (para 'Age')
df['Age'] = df['Age'].fillna(df['Age'].mean())

# Imputar valores categóricos com a moda (para 'Embarked')
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Verificar novamente os valores ausentes
print(df.isnull().sum())


Age              0
Cabin          369
Embarked         0
Fare             0
Name             0
Parch            0
PassengerId      0
Pclass           0
Sex              0
SibSp            0
Survived         0
Ticket           0
dtype: int64


In [23]:
# Garantir que as alterações sejam feitas no DataFrame original com loc
df.loc[:, 'Age'] = df['Age'].fillna(df['Age'].mean())
df.loc[:, 'Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Verificar novamente os valores ausentes
print(df.isnull().sum())


Age              0
Cabin          369
Embarked         0
Fare             0
Name             0
Parch            0
PassengerId      0
Pclass           0
Sex              0
SibSp            0
Survived         0
Ticket           0
dtype: int64


In [24]:
df['Cabin'] = df['Cabin'].fillna('Desconhecido')


In [25]:
# Exibir os valores faltantes após os tratamentos
print(df.isnull().sum())


Age            0
Cabin          0
Embarked       0
Fare           0
Name           0
Parch          0
PassengerId    0
Pclass         0
Sex            0
SibSp          0
Survived       0
Ticket         0
dtype: int64


## 3. Imputação com Estatísticas de Tendência Central:

In [26]:
# Imputar valores numéricos com a média
df['Age'] = df['Age'].fillna(df['Age'].mean())  # Exemplo para uma variável numérica

# Imputar valores categóricos com a moda
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])  # Exemplo para uma variável categórica


## 4. Imputação com Modelos Preditivos:


In [39]:
# Verifique se há valores ausentes em 'Age'
df['Age'].isnull().sum()

# Se houver valores ausentes, extraia os dados com 'Age' ausente
df_faltante = df[df['Age'].isnull()]

# Verifique se 'df_faltante' não está vazio
print(df_faltante.shape)  # Isso irá mostrar quantas linhas estão faltando valores em 'Age'

# Se houver linhas para prever, faça a imputação
if df_faltante.shape[0] > 0:
    X_faltante = df_faltante[['Pclass', 'Fare', 'SibSp']]  # Usar as mesmas variáveis para fazer a previsão
    df_faltante['Age'] = modelo.predict(X_faltante)
    df_final = pd.concat([df_completo, df_faltante])
else:
    print("Não há dados ausentes para imputar.")


(0, 12)
Não há dados ausentes para imputar.
